### Imports

In [52]:
import os
import pywt

import numpy as np

import mne
from mne.time_frequency import psd_welch
from mne.decoding import Scaler
from mne.filter import construct_iir_filter

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import cross_val_score

## Preprocessing

### Loading edf

In [3]:
file = "..\dataverse_files\h01.edf"
edfs_path = "..\dataverse_files"
manifest_path = "..\dataverse_files\MANIFEST.txt"

In [4]:
def load_patients_data(edfs_path):
    raw_patients_data = []
    
    edfs_file_names = [f for f in os.listdir(edfs_path) if f.endswith('.edf')]
    
    for file_name in edfs_file_names:
        path = edfs_path + '\\' + file_name 
        raw_data = mne.io.read_raw_edf(path, preload=True, verbose=False)
        raw_patients_data.append(raw_data)

    return raw_patients_data

In [5]:
raw_patients_data = load_patients_data(edfs_path)

In [6]:
len(raw_patients_data)

28

### EEG signals filtration

In [10]:
iir = construct_iir_filter(dict(order=6, ftype='butter', output='sos'), 50, None, 250, 'low')


IIR filter parameters
---------------------
Butterworth low zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 12 (effective, after forward-backward)
- Cutoff at 50.00 Hz: -6.02 dB



In [11]:
# low pass
filtered_patients_data = [raw_patient_data.copy()
                          .filter(l_freq=None, h_freq=None, picks='eeg', method='iir', iir_params=iir, n_jobs=-1, verbose=False) 
                          for raw_patient_data in raw_patients_data]

In [12]:
raw_patients_data[0].to_data_frame().head(5)

,time,Fp2,F8,T4,T6,O2,Fp1,F7,T3,T5,O1,F4,C4,P4,F3,C3,P3,Fz,Cz,Pz
0,0,0.002500,0.002500,0.002500,0.00250,0.002500,0.0025,0.0025,0.002500,0.002500,0.00250,0.0025,0.002500,0.0025,0.0025,0.002500,0.00250,0.0025,0.0025,0.002500
1,4,0.002500,0.002500,0.002500,0.00250,0.002500,0.0025,0.0025,0.002500,0.002500,0.00250,0.0025,0.002500,0.0025,0.0025,0.002500,0.00250,0.0025,0.0025,0.002500
2,8,0.002500,0.002500,0.002500,0.00250,0.002500,0.0025,0.0025,0.002500,0.002500,0.00250,0.0025,0.002500,0.0025,0.0025,0.002500,0.00250,0.0025,0.0025,0.002500
3,12,0.461215,0.461215,0.308310,0.30831,0.155405,0.0025,0.0025,-0.150405,-0.150405,0.00250,0.0025,0.002500,0.0025,0.0025,-0.150405,-0.30331,0.0025,0.0025,-0.303310
4,16,0.461215,0.461215,0.461215,0.30831,0.155405,0.0025,0.0025,-0.150405,-0.150405,-0.30331,0.0025,0.155405,0.0025,0.0025,-0.150405,-0.30331,0.0025,0.0025,-0.150405


In [78]:
epochs = [
    mne.make_fixed_length_epochs(raw_patient, duration=25, preload=True) for raw_patient in raw_patients_data
];

Not setting metadata
37 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 37 events and 6250 original time points ...
0 bad epochs dropped
Not setting metadata
36 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 36 events and 6250 original time points ...
0 bad epochs dropped
Not setting metadata
36 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 36 events and 6250 original time points ...
0 bad epochs dropped
Not setting metadata
37 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 37 events and 6250 original time points ...
0 bad epochs dropped
Not setting metadata
37 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 37 events and 6250 original time points 

In [80]:
filtered_patients_data = [epoch.get_data() for epoch in epochs]

#### FB

In [73]:
c = 0.8679
dec_lo, dec_hi, rec_lo, rec_hi = [c, c], [-c, c], [c, c], [c, -c]
filter_bank = [dec_lo, dec_hi, rec_lo, rec_hi]
myWavelet = pywt.Wavelet(name="rmsrfs", filter_bank=filter_bank)

In [77]:
a = myWavelet.wavefun()
a

[array([0.        , 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15083324, 5.15083324,
        5.15083324, 5.15083324, 5.15083324, 5.15

In [70]:
t = pywt.orthogonal_filter_bank(x);
t

(array([1.37972055e-03, 1.37904719e-03, 1.37837382e-03, ...,
        1.34672577e-06, 6.73362884e-07, 0.00000000e+00]),
 array([-0.00000000e+00,  6.73362884e-07, -1.34672577e-06, ...,
         1.37837382e-03, -1.37904719e-03,  1.37972055e-03]),
 array([0.00000000e+00, 6.73362884e-07, 1.34672577e-06, ...,
        1.37837382e-03, 1.37904719e-03, 1.37972055e-03]),
 array([ 1.37972055e-03, -1.37904719e-03,  1.37837382e-03, ...,
        -1.34672577e-06,  6.73362884e-07, -0.00000000e+00]))

### Scaling EEG signals with Scaler from MNE

In [77]:
def mne_std_scaler(edf):
    scaler = Scaler(scalings='mean')
    return [scaler.fit_transform(patient_data.get_data()) for patient_data in edf]

In [84]:
def mne_robust_scaler(edf):
    scaler = Scaler(scalings='median')
    return [scaler.fit_transform(patient_data.get_data()) for patient_data in edf]

In [85]:
mne_std_scaled_patient_data = mne_robust_scaler(filtered_patients_data)

In [86]:
mne_std_scaled_patient_data[0]

array([[[-2.11758237e-21],
        [ 2.03597831e-01],
        [ 1.51966248e+00],
        ...,
        [ 1.11173074e-21],
        [ 3.81164826e-21],
        [ 8.47032947e-22]],

       [[ 4.23516474e-22],
        [-5.67331033e-02],
        [ 1.34360667e+00],
        ...,
        [-5.82335151e-22],
        [-1.27054942e-21],
        [-1.27054942e-21]],

       [[ 4.23516474e-22],
        [-6.75909435e-01],
        [ 3.66261108e-01],
        ...,
        [-5.82335151e-22],
        [-1.05879118e-22],
        [-6.08804931e-22]],

       ...,

       [[-9.52912066e-22],
        [ 0.00000000e+00],
        [-3.83627015e-01],
        ...,
        [ 3.70576914e-22],
        [ 4.23516474e-22],
        [ 1.05879118e-21]],

       [[-2.96461532e-21],
        [-4.88317916e-02],
        [-5.15631602e-01],
        ...,
        [ 4.49986253e-22],
        [ 8.47032947e-22],
        [ 7.41153829e-22]],

       [[ 3.81164826e-21],
        [-2.72861688e-01],
        [-1.29895388e+00],
        ...,
        

### Filtered EEG signals segmentation

In [9]:
def get_label(edf):
    patient_edf_file_name = edf.filenames[0].split('\\')[-1]
    isSick = patient_edf_file_name.lower().startswith('s')
    return int(isSick == True) # 1 - is sick, 0 is healthy

In [10]:
def print_info(epochs_num_per_patient, labels):
    print('\nEpochs number per patient: ', epochs_num_per_patient)
    
    class_0_num = sum(labels) 
    class_1_num = len(labels)-sum(labels)

    print('\nnegative: ', class_0_num)
    print('positive: ', class_1_num)

In [11]:
def transform_patients_data_into_X_y_sets(patients_data, info=True):
    epochs_per_patient = []
    labels = []
    
    epochs_num_per_patient = []
    for edf in raw_patients_data:
        epochs = mne.make_fixed_length_epochs(edf, duration=25, preload=True, verbose=False)
        
        epochs_per_patient.append(epochs)
        epochs_num_per_patient.append(len(epochs))
        
        label = get_label(edf)
        labels.extend([label for epoch in epochs])
    
    epochs = mne.concatenate_epochs(epochs_per_patient)
    
    if info:
        print_info(epochs_num_per_patient, labels)

    return (epochs, labels) # (X, y)

In [12]:
X, y = transform_patients_data_into_X_y_sets(filtered_patients_data)

Not setting metadata
1142 matching events found
No baseline correction applied
0 bad epochs dropped

Epochs number per patient:  [37, 36, 36, 37, 37, 37, 36, 36, 36, 44, 36, 36, 38, 34, 33, 45, 38, 48, 35, 29, 53, 36, 47, 34, 54, 43, 45, 86]

negative:  626
positive:  516


### Feature extraction

In [13]:
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5]
        Transformed data.
    """
    # specific frequency bands
    FREQ_BANDS = {"delta": [0.5, 4.5],
                  "theta": [4.5, 8.5],
                  "alpha": [8.5, 11.5],
                  "sigma": [11.5, 15.5],
                  "beta": [15.5, 30]}

    psds, freqs = psd_welch(epochs, picks='eeg', fmin=0.5, fmax=30.)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

In [14]:
features = eeg_power_band(X)

Effective window size : 1.024 (s)


In [15]:
features.shape

(1142, 95)

In [16]:
len(features[0])

95

### Classification

##### Without scaling

In [26]:
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.33, shuffle=True, random_state=41)

pipe = make_pipeline(
    RandomForestClassifier(n_estimators=100, random_state=42),
)

# Train
pipe.fit(X_train, y_train)

# Test
y_pred = pipe.predict(X_test)

# Assess the results
acc = accuracy_score(y_test, y_pred)

print("Accuracy score: {}".format(acc))

Accuracy score: 0.9602122015915119


##### With MinMaxScaler

In [27]:
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.33, shuffle=True, random_state=41)

pipe = make_pipeline(
    MinMaxScaler(copy=False),
    RandomForestClassifier(n_estimators=100, random_state=42),
)

# Train
pipe.fit(X_train, y_train)

# Test
y_pred = pipe.predict(X_test)

# Assess the results
acc = accuracy_score(y_test, y_pred)

print("Accuracy score: {}".format(acc))

Accuracy score: 0.9602122015915119


##### With MinMaxScaler

In [28]:
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.33, shuffle=True, random_state=41)

pipe = make_pipeline(
    StandardScaler(copy=False),
    RandomForestClassifier(n_estimators=100, random_state=42),
)

# Train
pipe.fit(X_train, y_train)

# Test
y_pred = pipe.predict(X_test)

# Assess the results
acc = accuracy_score(y_test, y_pred)

print("Accuracy score: {}".format(acc))

Accuracy score: 0.9602122015915119


##### With RobustScaler

In [30]:
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.33, shuffle=True, random_state=41)

pipe = make_pipeline(
    RobustScaler(copy=False),
    RandomForestClassifier(n_estimators=100, random_state=42),
)

# Train
pipe.fit(X_train, y_train)

# Test
y_pred = pipe.predict(X_test)

# Assess the results
acc = accuracy_score(y_test, y_pred)

print("Accuracy score: {}".format(acc))

Accuracy score: 0.9602122015915119


### Cross Validatated Classification

In [20]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
scores = cross_val_score(clf, features, y, cv=5)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.66 accuracy with a standard deviation of 0.03
